In [1]:
import Random
using Plots
using Statistics
using Printf

To do:
   1. Separate obtaining the positive energy differences into a different function since this needs to be run
      only once before the loop over beta values starts.

In [2]:
const seed = Random.seed!(42)  # Fixing the overall seed for reproducible results by running the entire script.

const length::Int32 = 30  # Square root of total number of spins in the square lattice
const spins::Int32 = length^2
const configurations::Int32 = 10000  # Total number of configurations to be obtained
const skip::Int32 = 100  # Configurations to be skipped
const temperatures::Int32 = 100
beta_values = Float64[i/temperatures for i in 0:temperatures]


function get_positive_energies()
    ΔE = sort(unique([2*mid*(left + right + above + below) for mid in [-1, 1], left in [-1, 1], right in [-1, 1], 
                    above in [-1, 1], below in [-1, 1]]))
    positive_ΔE = Int8[ΔE for ΔE in ΔE if ΔE > 0]
    return positive_ΔE
end


function get_acceptance_probabilities(beta::Float64, positive_ΔE::Array{Int8, 1})
    return Float64[exp(-beta*ΔE) for ΔE in positive_ΔE]
end

function mc_step!(state::Array{Int8, 2}, positive_ΔE::Array{Int8, 1}, acceptance_probabilities::Array{Float64, 1})
    rows = rand(seed, 1:length, skip*spins)
    cols = rand(seed, 1:length, skip*spins)
    for (row, col) in zip(rows, cols)
        ΔE = 2 * state[row, col] * (state[mod1(row+1, length), col] + state[row, mod1(col+1, length)] + 
                                    state[mod1(row-1, length), col] + state[row, mod1(col-1, length)])
        if ΔE > 0 && rand(seed) > acceptance_probabilities[findfirst(isequal(ΔE), positive_ΔE)]
            continue
        end
        state[row, col] *= -1
    end
    return
end

function ising2d(beta_values::Array{Float64, 1})
    mean_magnetization = Float64[]
    positive_ΔE::Array{Int8, 1} = get_positive_energies()
    for beta in beta_values
        start = time()
        println("β = ", beta)
        state = -ones(Int8, (length, length))
        acceptance_probabilities::Array{Float64, 1} = get_acceptance_probabilities(beta, positive_ΔE)
        magnetisation = sum(state)/spins  # Calculate the value of the magnetisation apart so that it is immutable.
        magnetisation_values = Float64[magnetisation]
        for _ in 1:configurations
            mc_step!(state, positive_ΔE, acceptance_probabilities)
            magnetisation = sum(state)/spins
            push!(magnetisation_values, magnetisation)
        end
        mean_value = abs(mean(magnetisation_values))
        push!(mean_magnetization, mean_value)

        elapsed = time() - start
        println(elapsed)
    end
    return mean_magnetization
end

ising2d (generic function with 1 method)

In [ ]:
mean_magnetization = ising2d(beta_values)

plot(beta_values, mean_magnetization)

β = 0.0
13.487766981124878
β = 0.01
13.674370050430298
β = 0.02
14.160507917404175
β = 0.03
14.367439031600952
β = 0.04
14.760193109512329
β = 0.05
15.157272100448608
β = 0.06
15.379553079605103
β = 0.07
15.768898010253906
β = 0.08
15.93838095664978
β = 0.09
16.215439081192017
β = 0.1
16.493194818496704
β = 0.11
16.748537063598633
β = 0.12
17.017290115356445
β = 0.13
17.334167003631592
β = 0.14
17.6227970123291
β = 0.15
17.87809681892395
β = 0.16
18.272588968276978
β = 0.17


In [ ]:
beta_values = Float64[i/temperatures for i in 0:temperatures]
typeof(beta_values)